<a href="https://colab.research.google.com/github/VenkateshDas/legal_text_retrieval_coliee/blob/master/basic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
file_path = '/content/drive/My Drive/COLIEE Retrieval /Data/statute_law_parsed_df.pkl'

In [0]:
import pandas
import pickle 

f = open(file_path,'rb')
df = pickle.load(f)
dataset = df['Article']

In [0]:
df.head()

,Number,ID,Article
0,1,en_pt1ch1at1,(Fundamental Principles)Article 1 Private righ...
1,2,en_pt1ch1at2,(Standards for Construction)Article 2 This Cod...
2,3,en_pt1ch2sc1at1,Article 3 The enjoyment of private rights comm...
3,3_2,en_pt1ch2sc2at1,Article 3-2 If the person making a juridical a...
4,4,en_pt1ch2sc3at1,(Age of Majority)Article 4 The age of majority...


In [0]:
from gensim import corpora
from nltk.tokenize import word_tokenize
import nltk 
nltk.download('punkt')
import pprint

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
print(stop_words)

{"she's", 'more', "you'll", 'doing', 'herself', 'under', 'those', 'where', 'hers', 'further', 'yours', 'mustn', 'not', 'were', 'be', 'ours', "weren't", 'your', 'when', 'but', 'of', 'after', 'on', 'with', 'no', 'other', 'll', 'for', 'both', 'own', 'can', "mightn't", 'if', 'too', 'did', 'wouldn', 'his', 'himself', 'aren', 'i', 'above', 'been', 'before', 'or', 'our', 'these', 'yourselves', 'down', 'm', 'by', 'her', 'haven', 'there', 'from', 'do', 'any', "doesn't", "haven't", 'who', 'am', 'during', 'was', 'here', 'mightn', "hadn't", 'isn', 'each', 'we', "needn't", 'shan', 'a', 'd', 'because', 'between', 'me', 'their', 'about', 'through', "didn't", 'you', "don't", 'the', "isn't", 'my', 'below', 'an', 'to', 'all', 'hadn', 'until', 'ain', 'that', 's', 'why', 'only', 'how', 'than', 'ourselves', "you'd", 'into', 'will', 'them', 'are', 'itself', 'having', "wouldn't", 're', 'nor', 'is', 'whom', 'couldn', 'o', 'have', 'now', 'just', 'weren', 'as', 'this', 'at', 'being', 'should', 'up', 'most', 'do

In [0]:
token = [word_tokenize(article.lower()) for article in dataset]
tokens = []
for l in token:
    s = [] 
    for w in l:
        if w not in stop_words: 
            s.append(w)
    tokens.append(s)
         

In [0]:
print(tokens[0])
print(len(tokens[0]))

['(', 'fundamental', 'principles', ')', 'article', '1', 'private', 'rights', 'must', 'congruent', 'public', 'welfare', '.', 'exercise', 'rights', 'performance', 'duties', 'must', 'done', 'good', 'faith', '.', 'abuse', 'rights', 'permitted', '.']
26


In [0]:
val = 0
total = []
for l in tokens:
    val += len(l)
    for i in l:
        total.append(i)

unique_words = set(total)

print(val)
print(len(total))
print(len(unique_words))

38367
38367
3162


In [0]:
art_dict = corpora.Dictionary(tokens)
print(art_dict.token2id)
print(len(art_dict))

{'(': 0, ')': 1, '.': 2, '1': 3, 'abuse': 4, 'article': 5, 'congruent': 6, 'done': 7, 'duties': 8, 'exercise': 9, 'faith': 10, 'fundamental': 11, 'good': 12, 'must': 13, 'performance': 14, 'permitted': 15, 'principles': 16, 'private': 17, 'public': 18, 'rights': 19, 'welfare': 20, '2': 21, 'code': 22, 'construction': 23, 'construed': 24, 'dignity': 25, 'equality': 26, 'essential': 27, 'honor': 28, 'individuals': 29, 'sexes': 30, 'standards': 31, ',': 32, '3': 33, 'applicable': 34, 'birth': 35, 'commences': 36, 'enjoy': 37, 'enjoyment': 38, 'foreign': 39, 'laws': 40, 'nationals': 41, 'otherwise': 42, 'prohibited': 43, 'regulations': 44, 'treaties': 45, 'unless': 46, '3-2': 47, 'act': 48, 'capacity': 49, 'intention': 50, 'juridical': 51, 'making': 52, 'manifesting': 53, 'mental': 54, 'person': 55, 'relevant': 56, 'void': 57, '20': 58, '4': 59, 'age': 60, 'majority': 61, 'years': 62, "'s": 63, '5': 64, ';': 65, 'acquiring': 66, 'acts': 67, 'applies': 68, 'apply': 69, 'consent': 70, 'contr

In [0]:
for token, token_id in art_dict.token2id.items():
    art_dict.id2token[token_id] = token

print(art_dict.id2token)
print(len(art_dict))

{0: '(', 1: ')', 2: '.', 3: '1', 4: 'abuse', 5: 'article', 6: 'congruent', 7: 'done', 8: 'duties', 9: 'exercise', 10: 'faith', 11: 'fundamental', 12: 'good', 13: 'must', 14: 'performance', 15: 'permitted', 16: 'principles', 17: 'private', 18: 'public', 19: 'rights', 20: 'welfare', 21: '2', 22: 'code', 23: 'construction', 24: 'construed', 25: 'dignity', 26: 'equality', 27: 'essential', 28: 'honor', 29: 'individuals', 30: 'sexes', 31: 'standards', 32: ',', 33: '3', 34: 'applicable', 35: 'birth', 36: 'commences', 37: 'enjoy', 38: 'enjoyment', 39: 'foreign', 40: 'laws', 41: 'nationals', 42: 'otherwise', 43: 'prohibited', 44: 'regulations', 45: 'treaties', 46: 'unless', 47: '3-2', 48: 'act', 49: 'capacity', 50: 'intention', 51: 'juridical', 52: 'making', 53: 'manifesting', 54: 'mental', 55: 'person', 56: 'relevant', 57: 'void', 58: '20', 59: '4', 60: 'age', 61: 'majority', 62: 'years', 63: "'s", 64: '5', 65: ';', 66: 'acquiring', 67: 'acts', 68: 'applies', 69: 'apply', 70: 'consent', 71: 'c

In [0]:
for text in tokens:
    print(len(text))
    print(text)

26
['(', 'fundamental', 'principles', ')', 'article', '1', 'private', 'rights', 'must', 'congruent', 'public', 'welfare', '.', 'exercise', 'rights', 'performance', 'duties', 'must', 'done', 'good', 'faith', '.', 'abuse', 'rights', 'permitted', '.']
16
['(', 'standards', 'construction', ')', 'article', '2', 'code', 'must', 'construed', 'honor', 'dignity', 'individuals', 'essential', 'equality', 'sexes', '.']
24
['article', '3', 'enjoyment', 'private', 'rights', 'commences', 'birth', '.', 'unless', 'otherwise', 'prohibited', 'applicable', 'laws', ',', 'regulations', ',', 'treaties', ',', 'foreign', 'nationals', 'enjoy', 'private', 'rights', '.']
16
['article', '3-2', 'person', 'making', 'juridical', 'act', 'mental', 'capacity', 'manifesting', 'relevant', 'intention', ',', 'juridical', 'act', 'void', '.']
12
['(', 'age', 'majority', ')', 'article', '4', 'age', 'majority', '20', 'years', 'age', '.']
79
['(', 'juridical', 'acts', 'minors', ')', 'article', '5', 'minor', 'must', 'obtain', 'co

In [0]:
i = 0
for token in tokens[0]:
    i += 1
    print(art_dict.token2id[token])
print(i)

0
11
16
1
5
3
17
19
13
6
18
20
2
9
19
14
8
13
7
12
10
2
4
19
15
2
26


In [0]:
#apply preprocessing before calling this method 

corpus = [art_dict.doc2bow(text) for text in tokens]
print(corpus)

[[(0, 1), (1, 1), (2, 3), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 3), (20, 1)], [(0, 1), (1, 1), (2, 1), (5, 1), (13, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1)], [(2, 2), (5, 1), (17, 2), (19, 2), (32, 3), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1)], [(2, 1), (5, 1), (32, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 2), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1)], [(0, 1), (1, 1), (2, 1), (5, 1), (58, 1), (59, 1), (60, 3), (61, 2), (62, 1)], [(0, 2), (1, 2), (2, 4), (3, 1), (5, 1), (13, 1), (15, 2), (32, 4), (48, 3), (51, 4), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 3), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 3), (79, 1), (80, 1), (81, 6), (82, 1), (83, 1), (84, 1), (85, 1), (86, 2)

In [0]:
print(len(corpus))
print(len(corpus[0]))
pprint.pprint(corpus[0])

782
21
[(0, 1),
 (1, 1),
 (2, 3),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 2),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 3),
 (20, 1)]


In [0]:
word_counts = [[(art_dict[id], count) for id, count in line] for line in corpus]
sum = 0
for id,count in corpus[0]:
    sum += count
print(sum)
pprint.pprint(word_counts[0])
print(tokens[0])

26
[('(', 1),
 (')', 1),
 ('.', 3),
 ('1', 1),
 ('abuse', 1),
 ('article', 1),
 ('congruent', 1),
 ('done', 1),
 ('duties', 1),
 ('exercise', 1),
 ('faith', 1),
 ('fundamental', 1),
 ('good', 1),
 ('must', 2),
 ('performance', 1),
 ('permitted', 1),
 ('principles', 1),
 ('private', 1),
 ('public', 1),
 ('rights', 3),
 ('welfare', 1)]
['(', 'fundamental', 'principles', ')', 'article', '1', 'private', 'rights', 'must', 'congruent', 'public', 'welfare', '.', 'exercise', 'rights', 'performance', 'duties', 'must', 'done', 'good', 'faith', '.', 'abuse', 'rights', 'permitted', '.']


In [0]:
from gensim import models
import numpy as np

#apply preprocessing

tfidf = models.TfidfModel(corpus, smartirs='ntc')
corpus_tfidf = tfidf[corpus]

for doc in tfidf[corpus]:
    print([[art_dict[id], np.around(freq, decimals=2)] for id, freq in doc])


[['(', 0.0], [')', 0.0], ['.', 0.0], ['1', 0.11], ['abuse', 0.29], ['article', 0.0], ['congruent', 0.33], ['done', 0.29], ['duties', 0.25], ['exercise', 0.14], ['faith', 0.16], ['fundamental', 0.29], ['good', 0.17], ['must', 0.18], ['performance', 0.11], ['permitted', 0.24], ['principles', 0.33], ['private', 0.27], ['public', 0.16], ['rights', 0.28], ['welfare', 0.33]]
[['(', 0.0], [')', 0.0], ['.', 0.0], ['article', 0.0], ['must', 0.09], ['2', 0.14], ['code', 0.24], ['construction', 0.2], ['construed', 0.33], ['dignity', 0.33], ['equality', 0.33], ['essential', 0.33], ['honor', 0.33], ['individuals', 0.33], ['sexes', 0.33], ['standards', 0.33]]
[['.', 0.0], ['article', 0.0], ['private', 0.49], ['rights', 0.17], [',', 0.03], ['3', 0.17], ['applicable', 0.23], ['birth', 0.29], ['commences', 0.19], ['enjoy', 0.25], ['enjoyment', 0.29], ['foreign', 0.21], ['laws', 0.16], ['nationals', 0.29], ['otherwise', 0.15], ['prohibited', 0.26], ['regulations', 0.17], ['treaties', 0.29], ['unless', 0

In [0]:
from gensim import similarities

index_tfidf = similarities.SparseMatrixSimilarity(corpus_tfidf, num_features=len(art_dict))

print(index_tfidf)

In [0]:
def search(index, query, top_n=5, prints=False):
    """
    This function searches the most similar texts to the query.
        :param index: gensim.similarities object
        :param query: a string
        :param top_n: how many variants it returns
        :param prints: if True returns the results, otherwise prints the results
        :returns: a list of tuples (matched_document_index, similarity_value)
    """
    # getting a BoW vector
    tokens = word_tokenize(query.lower())
    query_token = []
    for w in tokens:
        if w not in stop_words:
            query_token.append(w)
    bow_vec = art_dict.doc2bow(query_token)
    similarities = index[bow_vec]  # get similarities between the query and all index documents
    similarities = [(x, i) for i, x in enumerate(similarities)]
    similarities.sort(key=lambda elem: -elem[0])  # sorting by similarity_value in decreasing order
    res = []
    if prints:
        print("Query term")
        print(f"{query}\n")
    for result in similarities[:top_n]:
        if prints:
            print("*"*50)
            print(f"{df['Number'][result[1]]}\t {result[0]}\n ")
            print(f"{df['ID'][result[1]]}" ) #\t {result[0]}\n")
            print(f"{df['Article'][result[1]]}")# \t {result[0]}\n")
            print("*"*50)
        else:
            res.append((result[1], result[0]))
    if not prints:
        return res

In [0]:
query = ''' 

With respect to any inherited property, even if the administrator of the inherited property is appointed, the prescription shall not be completed until the applicable heir is identified.
'''

results = search( index_tfidf , query , 5 , prints=False)

print(results)

[(115, 0.27099884), (25, 0.2087162), (27, 0.20435093), (191, 0.19824505), (29, 0.19313952)]


The articles in the Dataset given slightly varies from the downloaded Civil code from the website 

"Any person who has become the subject of the ***adjudication of disappearance*** pursuant to the provision of paragraph 1 of the preceding Article is ***deemed to have died upon elapse of the period set forth in such paragraph***, and a person who is the subject of the adjudication of disappearance pursuant to the provision of paragraph 2 of the same Article is deemed to have died upon the*** termination of such danger***."


"(Effect of Declarations of Disappearance)Article 31 A person subject to a ***declaration of disappearance pursuant*** to the provisions of paragraph (1) of the preceding Article is deemed to have died ***when the period of time referred to in that paragraph ended***, and a person subject to a declaration of disappearance pursuant to the provisions of paragraph (2) of that Article is deemed to have*** died when that danger had passed***."